In [2]:
!ls /media/mengjie/kaldi/egs/irish_2021/s5/exp_addweb2607_cgm/chain/tdnn1i_sp_ep10/graph_cgm_trwp0.3gaois3gr

disambig_tid.int  mkgraph.log  phones	   words.txt
HCLG.fst	  num_pdfs     phones.txt


In [7]:
!cat /media/mengjie/kaldi/egs/irish_2021/s5/exp_addweb2607_cgm/chain/tdnn1i_sp_ep10/graph_cgm_trwp0.3gaois3gr/mkgraph.log

# utils/mkgraph.sh --self-loop-scale 1.0 data/lang_cgm_trwp0.3gaois3gr exp_addweb2607_cgm/chain/tdnn1i_sp_ep10 exp_addweb2607_cgm/chain/tdnn1i_sp_ep10/graph_cgm_trwp0.3gaois3gr 
# Started at Sun Mar 14 23:32:34 GMT 2021
#
tree-info exp_addweb2607_cgm/chain/tdnn1i_sp_ep10/tree 
tree-info exp_addweb2607_cgm/chain/tdnn1i_sp_ep10/tree 
fstminimizeencoded 
fstdeterminizestar --use-log=true 
fsttablecompose data/lang_cgm_trwp0.3gaois3gr/L_disambig.fst data/lang_cgm_trwp0.3gaois3gr/G.fst 
fstpushspecial 
fstisstochastic data/lang_cgm_trwp0.3gaois3gr/tmp/LG.fst 
-0.105623 -0.106012
[info]: LG not stochastic.
fstcomposecontext --context-size=2 --central-position=1 --read-disambig-syms=data/lang_cgm_trwp0.3gaois3gr/phones/disambig.int --write-disambig-syms=data/lang_cgm_trwp0.3gaois3gr/tmp/disambig_ilabels_2_1.int data/lang_cgm_trwp0.3gaois3gr/tmp/ilabels_2_1.14178 data/lang_cgm_trwp0.3gaois3gr/tmp/LG.fst 
fstisstochastic data/lang_cgm_trwp0.3gaois3gr/tmp/CLG_2_1.fst 
-0.105623 -0.106012
[info]:

In [3]:
!find /media/mengjie/kaldi/egs/irish_2021/s5/ -name '*sh'|xargs grep trwp

find: ‘/media/mengjie/kaldi/egs/irish_2021/s5/exp_addweb2607_gd/nnet3/diag_ubm/backup.cut’: Permission denied
find: ‘/media/mengjie/kaldi/egs/irish_2021/s5/exp_addweb2607_glb/nnet3/diag_ubm/backup.ieZ’: Permission denied
find: ‘/media/mengjie/kaldi/egs/irish_2021/s5/exp_addweb2607/nnet3/diag_ubm/backup.us8’: Permission denied
find: ‘/media/mengjie/kaldi/egs/irish_2021/s5/exp_addweb2607_glbcm/nnet3/diag_ubm/backup.TCs’: Permission denied
find: ‘/media/mengjie/kaldi/egs/irish_2021/s5/exp_addweb2607_cgm/nnet3/diag_ubm/backup.Yqf’: Permission denied
find: ‘/media/mengjie/kaldi/egs/irish_2021/s5/exp_addweb2607_cgmg/nnet3/diag_ubm/backup.TrN’: Permission denied
find: ‘/media/mengjie/kaldi/egs/irish_2021/s5/exp_addweb2607_cm/nnet3/diag_ubm/backup.wxT’: Permission denied
find: ‘/media/mengjie/kaldi/egs/irish_2021/s5/exp_addweb2607_mu/nnet3/diag_ubm/backup.NXQ’: Permission denied
/media/mengjie/kaldi/egs/irish_2021/s5/run_temp.sh:# 	--ac-model-dir exp_addweb2607_glb/chain/tdnn1i_sp_ep10 --lex-a

In [4]:
!cat /media/mengjie/kaldi/egs/irish_2021/s5/run_20210312_train_lm.sh

#!/bin/bash -u
stage=0



#################################### prepare language model (arpa file) ########################################
echo "############ Compare multiple arpa later! ############"
if [ $stage -le 4 ];then
  ngr=3
  oov_symbol="<unk>"
  arpadir=data/local/srilm
  mkdir -p $arpadir
  train=data/train_addweb2607
  lm_train_text=$arpadir/train_addweb2607_text
  cut -d' ' -f2- $train/text > ${lm_train_text}
  ngram-count \
    -lm - -order ${ngr} -text ${lm_train_text} \
    -unk -sort -maxent -maxent-convert-to-arpa|\
    ngram -lm - -order ${ngr} -unk -map-unk "$oov_symbol" -prune-lowprobs -write-lm - |\
    sed 's/<unk>/'${oov_symbol}'/g' | gzip -c > ${arpadir}/train.${ngr}gram.gz
fi

# wp
if [ $stage -le 4 ];then
  cp ../../irish_2020/s5/data/lang_interp/wp_corpus_v3.txt data/local/
  ngr=3
  oov_symbol="<unk>"
  arpadir=data/local/srilm
  mkdir -p $arpadir
  lm_train_text=data/local/srilm/wp_corpus_v3.txt
  ngram-count \
    -lm - -order ${ngr} -text ${lm_train_tex

In [5]:
!cat /media/mengjie/kaldi/egs/irish_2021/s5/run_temp_gmm.sh

# decode
# lex_affix=""
# # for lm_affix in "_wp" "_gaois";do
# for lm_affix in "_trwp";do
# expdir=exp_addweb2607
# graph_affix=${lex_affix}${lm_affix}3gr
# bash run_decode_test.sh --stage 4 --data mozilla_set1 --expdir $expdir --graph-affix $graph_affix \
#   --tdnndir $expdir/chain/tdnn1i_sp_ep10  |& tee -a log/log_run_20210313_decode_${expdir}${graph_affix}_mozilla_set1
# done


# for w in $(seq 0.1 0.1 0.9);do 
#   lm_affix="_trwp"${w}"gaois"
#   expdir=exp_addweb2607
#   graph_affix=${lex_affix}${lm_affix}3gr
#   bash run_decode_test.sh --stage 4 --data mozilla_set1 --expdir $expdir --graph-affix $graph_affix \
#     --tdnndir $expdir/chain/tdnn1i_sp_ep10  |& tee -a log/log_run_20210313_decode_${expdir}${graph_affix}_mozilla_set1
# done

# expdir=exp_addweb2607
# graph_affix=_trwp4gr
# bash run_decode_test.sh --stage 4 --data mozilla_set1 --expdir $expdir --graph-affix $graph_affix \
#     --tdnndir $expdir/chain/tdnn1i_sp_ep10  |& tee -a log/log_run_20210313_decode_${expdir}${gr

In [6]:
!cat /media/mengjie/kaldi/egs/irish_2021/s5/run_gmm.sh

#!/bin/bahs
# Train GMM models for input dataset

# Begin configuration
stage=7
nj=8
train=data/cillairne
test=data/mozilla_set1
expdir=exp 
lang_affix="1"            # corresponding to the lexicon
graph_affix="tr_wp_0.5"   # corresponding to both the lexicon and the LM (arpa)
decode=true

pdf=4000
gauss=20000
pdf2b=5000
gauss2b=30000
pdf3b=8400
gauss3b=80000
# End configuration

echo "$0 $@"  # Print the command line for logging
. ./utils/parse_options.sh
. ./cmd.sh
. ./path.sh

lang=data/lang${lang_affix}
LM=data/lang${graph_affix}
test_set=`basename $test`


if [ $stage -le 0 ]; then
  steps/make_mfcc.sh $train
  steps/compute_cmvn_stats.sh $train
fi

if [ $stage -le 1 ]; then
    steps/train_mono.sh --nj $nj --cmd "$train_cmd" \
      $train $lang $expdir/mono || exit 1;

    steps/align_si.sh --nj $nj --cmd "$train_cmd" \
      $train $lang $expdir/mono $expdir/mono_ali || exit 1;
fi

if [ $stage -le 2 ]; then
  if $decode; then
    mkdir -p $expdir/mono/graph${graph_affix}
    ru

```
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 36
steps/train_mono.sh: Pass 37
steps/train_mono.sh: Pass 38
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 39
steps/diagnostic/analyze_alignments.sh --cmd run.pl data/lang exp/mono
steps/diagnostic/analyze_alignments.sh: see stats in exp/mono/log/analyze_alignments.log
152 warnings in exp/mono/log/update.*.log
196236 warnings in exp/mono/log/align.*.*.log
8587 warnings in exp/mono/log/acc.*.*.log
exp/mono: nj=10 align prob=-92.62 over 283.50h [retry=1.6%, fail=0.1%] states=142 gauss=988
steps/train_mono.sh: Done training monophone system in exp/mono
Making recognition graph for mono
mkgraph.sh: expected data/lang_test_4g/L_disambig.fst to exist
root@844ceced5e98:/opt/kaldi/egs/sb/s5# ls
RESULTS  cmd.sh  conf  data  exp  local  mfcc  path.sh  run.sh  steps  utils
```

```
steps/train_mono.sh --nj 10 --cmd "$train_cmd" data/train data/lang exp/mono || exit 1;

utils/mkgraph.sh data/lang_test_4g exp/mono exp/mono/graph_4g || exit 1;
```